In [1]:
import numpy as np
import cv2
import math

In [ ]:
def float_to_rgbe_and_process(image_float, exposure_bits=6):
    """
    将 float32 图像转换为 RGBE，处理 Exponent，然后再转回 float32。
    """
    height, width, channels = image_float.shape
    
    # 1. 准备 RGBE 容器 (uint8)
    # 这里的计算逻辑参考 Radiance 官方定义
    # 找出每个像素 RGB 中的最大值
    v = np.max(image_float, axis=2)
    
    # 避免 log2(0) 的错误，设置极小值
    v[v < 1e-32] = 1e-32
    
    # 2. 计算指数 (Exponent)
    # 利用 frexp 分解: value = mantissa * 2^exponent
    # mantissa 范围是 [0.5, 1)
    mantissa, exponent = np.frexp(v)
    
    # Radiance 标准偏移量是 128
    # E_standard = exponent + 128
    e_layer = exponent + 128
    
    # ==========================================
    # 核心操作：在这里把 E 砍到 6 bit
    # ==========================================
    
    # 6-bit 意味着范围是 0~63。
    # 我们需要定义一个新的 Bias (偏移量)。
    # 标准 8-bit 偏移 128。对于 6-bit，通常偏移量选中间值 32。
    # 这意味着我们能表达的真实指数范围是 [-32, 31]
    
    # 第一步：算出真实的 exponent
    real_exponent = e_layer - 128
    
    # 第二步：限制真实 exponent 在 6-bit 能表达的范围内
    # Min: -32, Max: 31
    # 任何比 2^-32 暗的会被切除，比 2^31 亮的会被切除
    bias_6bit = 32
    min_exp = -bias_6bit        # -32
    max_exp = (63 - bias_6bit)  # 31
    
    # 执行截断 (Clamping)
    real_exponent_clamped = np.clip(real_exponent, min_exp, max_exp)
    
    # 第三步：为了存回标准的 8-bit 格式，我们需要把偏移量加回去
    # 这样查看器才能正确理解我们模拟的亮度
    e_final = real_exponent_clamped + 128
    
    # 转换为 uint8
    e_final = e_final.astype(np.uint8)
    
    # ==========================================
    # 3. 计算 Mantissa (RGB 部分)
    # ==========================================
    
    # 公式: RGB_stored = RGB_float * (256 / 2^exponent)
    # 注意：这里的 exponent 必须是我们 Clamped 之后的
    # 因为我们改变了 E，必须重新计算对应的 RGB 尾数，否则颜色会错
    
    scale = np.ldexp(1.0, -real_exponent_clamped.astype(int)) * 256
    scale = np.expand_dims(scale, axis=2) # 扩展维度以便广播乘法
    
    rgb_final = image_float * scale
    
    # 限制在 0-255 并转为 uint8
    rgb_final = np.clip(rgb_final, 0, 255).astype(np.uint8)
    
    # 组合 RGB 和 E
    rgbe_image = np.dstack((rgb_final, e_final))
    
    return rgbe_image

In [3]:
def rgbe_to_float(rgbe_image):
    """
    将 RGBE 解码回 Float32 以便保存
    """
    r = rgbe_image[:, :, 0]
    g = rgbe_image[:, :, 1]
    b = rgbe_image[:, :, 2]
    e = rgbe_image[:, :, 3]

    # 解码公式: (Value + 0.5) / 256 * 2^(e - 128)
    # 为了简化，通常忽略 +0.5 的 dithering
    
    scale = np.ldexp(1.0, e.astype(int) - 128) / 256.0
    scale = np.expand_dims(scale, axis=2)
    
    rgb = np.dstack((r, g, b))
    image_float = rgb.astype(np.float32) * scale
    
    return image_float

In [ ]:
# 1. 读取原始 HDR
input_path = "./hdr_file/rogland_clear_night_1k.hdr" 
output_path = "output_6bit_E.hdr"

# flags=-1 确保以 float32 格式读取原始数据
img = cv2.imread(input_path, -1) 

print(f"原始图像范围: Min {np.min(img):.5f}, Max {np.max(img):.5f}")

# 2. 转为 RGBE 并模拟 6-bit E 的截断
rgbe_data = float_to_rgbe_and_process(img, exposure_bits=6)

# 3. 转回 Float 准备保存
img_processed = rgbe_to_float(rgbe_data)

print(f"处理后图像范围: Min {np.min(img_processed):.5f}, Max {np.max(img_processed):.5f}")
print("注意：如果原始图像非常亮或非常暗，Max/Min 就会被锁定在 6-bit 的极限值上。")

# 4. 保存为标准 HDR
# OpenCV 保存时会自动根据 float 数据重新生成标准的 8-bit RGBE
# 但由于我们的数据已经被 Clamp 过了，生成的 E 自然不会超出范围
cv2.imwrite(output_path, img_processed)

print(f"已保存到: {output_path}")

原始图像范围: Min 0.00342, Max 73.50000
处理后图像范围: Min 0.00342, Max 73.50000
注意：如果原始图像非常亮或非常暗，Max/Min 就会被锁定在 6-bit 的极限值上。
已保存到: output_6bit_E.hdr


In [9]:
import numpy as np

# ----------- 讀 RLE HDR (同先前版本) -----------
def read_hdr_rgbe(path):
    with open(path, "rb") as f:
        while True:
            line = f.readline().decode(errors="ignore")
            if line.strip() == "":
                break

        line = f.readline().decode().strip().split()
        H = int(line[1])
        W = int(line[3])

        img = np.zeros((H, W, 4), dtype=np.uint8)

        for y in range(H):
            header = f.read(4)
            if header[0]!=2 or header[1]!=2:
                raise ValueError("not RLE HDR")
            scan_width=(header[2]<<8)|header[3]
            scan=np.zeros((scan_width,4),dtype=np.uint8)

            for c in range(4):          # R  G  B  E
                x=0
                while x<scan_width:
                    val=ord(f.read(1))
                    if val>128:
                        cnt=val-128
                        b=ord(f.read(1))
                        scan[x:x+cnt,c]=b
                        x+=cnt
                    else:
                        raw=f.read(val)
                        scan[x:x+val,c]=list(raw)
                        x+=val
            img[y]=scan
    return img,W,H



# ----------- E clip 成 6 bit -----------
def clip_E_6bit(hdr):
    hdr2 = hdr.copy()
    hdr2[:,:,3] = np.clip(hdr[:,:,3], 0, 63)  # <= E 變 6 bit
    return hdr2



# ----------- 寫回 RLE HDR -----------
def write_hdr_rgbe(path, img,W,H):
    with open(path,"wb") as f:
        # Radiance Header
        f.write(b"#?RADIANCE\nFORMAT=32-bit_rle_rgbe\n\n")
        f.write(f"-Y {H} +X {W}\n".encode())

        for y in range(H):
            f.write(bytes([2,2,(W>>8)&255,W&255]))

            scan = img[y]

            for c in range(4):
                x=0
                while x<W:
                    run_len=1
                    while x+run_len<W and scan[x+run_len,c]==scan[x,c] and run_len<127:
                        run_len+=1

                    if run_len>=2:  # run block
                        f.write(bytes([128+run_len, scan[x,c]]))
                        x+=run_len
                    else:          # literal block
                        start=x
                        x+=1
                        while x<W and scan[x,c]!=scan[x-1,c] and (x-start)<127:
                            x+=1
                        block=scan[start:x,c].tolist()
                        f.write(bytes([len(block)]))
                        f.write(bytes(block))

    print("✔ HDR saved:",path)



# ----------- 使用範例 -----------
hdr,W,H = read_hdr_rgbe("./hdr_file/rogland_clear_night_1k.hdr")

hdr_6bit = clip_E_6bit(hdr)

write_hdr_rgbe("output_E6.hdr", hdr_6bit, W,H)


✔ HDR saved: output_E6.hdr


In [ ]:
import numpy as np

# ----------- 讀取 RLE HDR (前段同前，但保留方便你直接執行) -----------
def read_hdr_rgbe(path):
    with open(path, "rb") as f:
        while True:
            line = f.readline().decode(errors="ignore")
            if line.strip()=="":
                break

        line=f.readline().decode().strip().split()
        H=int(line[1])
        W=int(line[3])

        img=np.zeros((H,W,4),dtype=np.uint8)

        for y in range(H):
            header=f.read(4)
            if header[0]!=2 or header[1]!=2:
                raise ValueError("Not RLE Radiance HDR")

            scan = np.zeros((W,4),dtype=np.uint8)
            for c in range(4):
                x=0
                while x<W:
                    val=ord(f.read(1))
                    if val>128:   # run
                        cnt=val-128
                        b=ord(f.read(1))
                        scan[x:x+cnt,c]=b
                        x+=cnt
                    else:         # literal
                        raw=f.read(val)
                        scan[x:x+val,c]=list(raw)
                        x+=val
            img[y]=scan
    return img,W,H


# ----------- ★ 你的 exponent 6-bit clip 算法 ★ -----------
def clamp_E_6bit_signed(hdr):
    hdr2=hdr.copy().astype(np.int16)

    E = hdr2[:,:,3]                             # 0~255
    Esigned = E - 128                           # → -128 ~ +127
    Eclamp = np.clip(Esigned, -8, 7)          # → 限制到 6-bit signed
    Enew = (Eclamp + 128).astype(np.uint8)      # → 回到 RGBE 格式

    hdr2[:,:,3] = Enew
    return hdr2.astype(np.uint8)


# ----------- 寫回 HDR (RLE) -----------
def write_hdr_rgbe(path,img,W,H):
    with open(path,"wb") as f:
        f.write(b"#?RADIANCE\nFORMAT=32-bit_rle_rgbe\n\n")
        f.write(f"-Y {H} +X {W}\n".encode())

        for y in range(H):
            f.write(bytes([2,2,(W>>8)&255,W&255]))
            scan=img[y]

            for c in range(4):
                x=0
                while x<W:
                    # RLE run block
                    run=1
                    while x+run<W and scan[x+run,c]==scan[x,c] and run<127:
                        run+=1

                    if run>=2:
                        f.write(bytes([128+run,scan[x,c]]))
                        x+=run
                    else:
                        start=x
                        x+=1
                        while x<W and scan[x,c]!=scan[x-1,c] and (x-start)<127:
                            x+=1
                        block=scan[start:x,c].tolist()
                        f.write(bytes([len(block)]))
                        f.write(bytes(block))

    print("✔ saved:",path)


# ---------- 使用示例 ----------
hdr,W,H = read_hdr_rgbe("./hdr_file/symmetrical_garden_02_1k.hdr")

hdr6 = clamp_E_6bit_signed(hdr)

write_hdr_rgbe("symmetrical_garden_02_1k_clip.hdr", hdr6, W,H)


✔ saved: symmetrical_garden_02_1k_clip.hdr
